In [1]:
import matplotlib
%matplotlib tk
%autosave 180
%load_ext autoreload
%autoreload 2

import nest_asyncio
%config Completer.use_jedi = False

#
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# 
import numpy as np
import os
import scipy

# add root directory to be able to import packages
# todo: make all packages installable so they can be called/imported by environment
import sys
module_path = os.path.abspath(os.path.join('..'))
sys.path.append(module_path)

from utils.calcium import calcium


Autosaving every 180 seconds


<ipython-input-1-e4476ecac24a>:12: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


/home/cat/.conda/envs/donato/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
#############################################
######### LOAD SUITE2P AND BINARIZE #########
#############################################

# input directory where Suite2p outputs matlab Fall.mat and all .npy files
# load list of directories to be processed
root_dir = '/media/cat/4TB/donato/'
animal_id = 'DON-002865'

animal_ids = [
#'DON-002865',
'DON-003165',
'DON-003343',
'DON-006084',
'DON-006085',
'DON-006087',
]

for animal_id in animal_ids:
    sessions = os.listdir(os.path.join(root_dir, animal_id))

    # initialize calcium object and load suite2p data
    for session in sessions:
        print ("\n\n")
        print ("Processing session: ", session)

        # initialize calcium object and load suite2p data
        c = calcium.Calcium()
        c.verbose = False                          # outputs additional information during processing
        c.recompute_binarization = True           # recomputes binarization and other processing steps; False: loads from previous saved locations
        c.data_dir = os.path.join(root_dir,
                                  animal_id,
                                  session,
                                  '002P-F',
                                 'tif',
                                 'suite2p',
                                 'plane0')
        c.load_suite2p()                          

        # set flags to save matlab and python data
        c.save_python = True         # save output as .npz file 
        c.save_matlab = False         # save output as .mat file

        ###############################################
        ##### PARAMETERS FOR RUNNING BINARIZATION #####
        ###############################################
        c.min_width_event_onphase = c.sample_rate//2 # set minimum withd of an onphase event; default: 0.5 seconds
        c.min_width_event_upphase = c.sample_rate//4 # set minimum width of upphase event; default: 0.25 seconds

        ############# PARAMTERS TO TWEAK ##############
        #     1. Cutoff for calling somthing a spike:
        #        This is stored in: std_Fluorescence_onphase/uppohase: defaults: 1.5
        #                                        higher -> less events; lower -> more events
        #                                        start at default and increase if data is very noisy and getting too many noise-events
        c.min_thresh_std_onphase = 2.5      # set the minimum thrshold for onphase detection; defatul 2.5
        c.min_thresh_std_upphase = 2.5      # set the minimum thershold for uppohase detection; default: 2.5

        #     2. Filter of [Ca] data which smooths the data significantly more and decreases number of binarzied events within a multi-second [Ca] event
        #        This is stored in high_cutoff: default 0.5 to 1.0
        #        The lower we set it the smoother our [Ca] traces and less "choppy" the binarized traces (but we loose some temporal precision)
        c.high_cutoff = 0.5              

        #     3. Removing bleaching and drift artifacts using polynomial fits
        #        This is stored in detrend_model_order
        c.detrend_model_order = 1 # 1-3 polynomial fit
        c.detrend_model_type = 'mode' # 'mode', 'polynomial'

        #

        ################################################
        ########### RUN BINARIZATION STEP ##############
        ################################################
        # 
        # double check that we have set the STD thrshold at a reasonable level to catch biggest/highest SNR bursts
        c.show_plots = True

        #
        c.min_width_event_onphase = 30
        c.min_width_event_upphase = 10
        c.recompute_binarization = True

        # not used currently
        c.dff_min = 0.02   #this is the final threshold required by putative spikes
        c.percentile_threshold = 0.999999
        c.use_upphase = True

        #
        c.show_plots =False
        c.remove_ends = False
        c.detrend_filter_threshold = 0.001

        if animal_id == 'DON-006085':                           ################ SPECIAL CASE; THIS MOUSE HAS CORTICAL WAVES######
            c.mode_window = 30*30 #*30 # 120 seconds @ 
        else:
            c.mode_window = None #*30 # 120 seconds @ 
        c.binarize_fluorescence()


        # generate standard randomized plots:
        c.save_sample_traces()

        #
        save_image = True
        c.show_rasters(save_image)

print (" ... DONE ...")




Processing session:  DON-003165_20210220


model filter: remove bleaching or trends: 100%|██████████| 301/301 [00:01<00:00, 250.49it/s]
305it [00:02, 149.06it/s]                         
binarizing continuous traces filtered fluorescence onphase: 100%|██████████| 301/301 [00:00<00:00, 1745.07it/s]
binarizing continuous traces filtered fluorescence upphase: 100%|██████████| 301/301 [00:00<00:00, 460.40it/s]





Processing session:  DON-003165_20210216


model filter: remove bleaching or trends: 100%|██████████| 389/389 [00:01<00:00, 264.95it/s]
392it [00:02, 170.52it/s]                         
binarizing continuous traces filtered fluorescence onphase: 100%|██████████| 389/389 [00:00<00:00, 1763.50it/s]
binarizing continuous traces filtered fluorescence upphase: 100%|██████████| 389/389 [00:00<00:00, 468.94it/s]





Processing session:  DON-003165_20210219


...




Processing session:  20210524


100%|██████████| 244/244 [00:01<00:00, 200.26it/s]
binarizing continuous traces filtered fluorescence onphase: 100%|██████████| 244/244 [00:00<00:00, 1609.15it/s]
binarizing continuous traces filtered fluorescence upphase: 100%|██████████| 244/244 [00:00<00:00, 554.06it/s]





Processing session:  20210528


model filter: remove bleaching or trends: 100%|██████████| 764/764 [00:02<00:00, 277.38it/s]
768it [00:03, 200.58it/s]                         
binarizing continuous traces filtered fluorescence onphase: 100%|██████████| 764/764 [00:00<00:00, 1261.29it/s]
binarizing continuous traces filtered fluorescence upphase: 100%|██████████| 764/764 [00:01<00:00, 550.49it/s]


 ... DONE ...


In [117]:
###############################################################
######### LOAD BINARZIED DATA AND VISUALIZE CELL ACTIVITY #####
###############################################################
#fname = '/media/cat/4TB/donato/DON-003343/DON-003343_20210213/suite2p/plane0/binarized_traces.npz'
# # 
fname = os.path.join(data_dir,'suite2p','plane0','binarized_traces.npz')

c = calcium.Calcium()
c.fname = fname
c.data_dir = data_dir
c.recompute_binarization = False
c.load_binarization()

# pick a random cell to visualize
cell_id = 986
scale = .3

# 
c.plot_cell_binarization(cell_id, scale)


In [68]:
######################################################
############# SHOW RASTERS ###########################
######################################################
fname = os.path.join(data_dir,'suite2p','plane0','binarized_traces.npz')
#data_dir = '/media/cat/4TB/donato/DON-003343/DON-003343_20210213/suite2p/plane0/'

c = calcium.Calcium()
c.data_dir = data_dir
c.fname = fname
c.recompute_binarization = False
c.load_binarization()

c.show_rasters()
